### ***Simple Gen AI App Using Langchain***

- ***Get Keys***
- ***Ingestion - Scrape the Website***
- ***Apply Recursive Character Text Splitter***
- ***Use Open AI Embeddings to convert documents into Vectors and Store into Vector DB***
- ***Query Vector DB***

***RAG Pipeline***

***Flow: Query → Retriever → Context → Document Chain → LLM → Answer.***

***code sets up a retrieval‑augmented generation workflow where GPT‑4o answers questions using only retrieved documents as context.***

***`Its not working as I had imagined. The input gets messed up. Since the input gets messed up, Output is not what I am expecting. Need to look at this code in future. Also, instead of reading from website, I would like to modify this code to read from pdf, which will be more beneficial for my personal use, and replace open ai model with free models, to avoid the cost`***

***Get Keys***

In [1]:
import os # Importing the os module to interact with the operating system
from dotenv import load_dotenv # Importing load_dotenv function from dotenv module to load environment variables from a .env file
load_dotenv() # Load environment variables from a .env file into the system's environment variables

# Set environment variables for API keys and Langchain tracing
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

***Data Ingestion - Scrape the Website***

*Document Loaders - Load the data from different sources*

In [2]:
## Data Ingestion--From the website we need to scrape the data
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
loader = WebBaseLoader("https://www.geeksforgeeks.org/artificial-intelligence/what-is-artificial-intelligence-ai/")
docs = loader.load()

print(f"loader type: {type(loader)}")
print(f"Number of documents loaded: {len(docs)}")

loader type: <class 'langchain_community.document_loaders.web_base.WebBaseLoader'>
Number of documents loaded: 1


***Apply Recursive Character Text Splitter***

*To split the documents into smaller documents called chunks, instead of giving entire document to LLM Model*

In [4]:
### Load Data--> Docs-->Divide our Docuemnts into chunks dcouments-->text-->vectors-->Vector Embeddings--->Vector Store DB
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

documents = text_splitter.split_documents(docs)
print(f"Number of documents after splitting: {len(documents)}")

Number of documents after splitting: 21


In [5]:
# documents

***Use Open AI Embeddings to convert documents into Vectors and Store into FAISS Vector DB***

- *`OpenAIEmbeddings` is LangChain’s wrapper around OpenAI’s embedding models (like text-embedding-3-small and text-embedding-3-large). By default, it uses `text-embedding-3-small` unless you specify another model:*
- *It provides a simple interface to generate vector representations of text that can be used for semantic search, clustering, or retrieval‑augmented generation (RAG).*
- *Every call to OpenAIEmbeddings sends text to OpenAI’s API and is billed per token.*

In [6]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings() # creates an instance of the embedding model

*Each Document's text is converted into a vector using the embedding models. FAISS indexes those vectors internally and eventually creates `vectorstoredb` that is searchable database which supports similarity search*

- *`FAISS` is LangChain’s integration with Facebook AI Similarity Search (FAISS), a library for efficient similarity search on dense vectors.*
- *`documents`: a list of LangChain Document objects (each has page_content and optional metadata).*
- *`embeddings`: your OpenAIEmbeddings instance*

In [7]:
from langchain_community.vectorstores import FAISS
vectorstoredb = FAISS.from_documents(documents,embeddings)

In [8]:
type(vectorstoredb)

langchain_community.vectorstores.faiss.FAISS

***Query Vector DB***

*Similarity Search finds relevant context. It returns documents that are closed in vector space to the query. Its purely mathematical, cosine similarity, dot product or Euclidean distance between embeddings*

In [9]:
## Query From a vector db
query = "What are Expert Systems?"
result = vectorstoredb.similarity_search(query)
result[0].page_content

'understand and generate human language allowing for tasks like language translation, sentiment analysis and real-time conversation.4. Expert SystemsExpert Systems are designed to simulate the decision-making ability of human experts. These systems use a set of predefined "if-then" rules and knowledge from specialists in specific fields to make informed decisions similar to how a medical professional would diagnose a disease. They are useful in areas where expert knowledge is important but not always easily accessible. Working of Artificial IntelligenceAI works by simulating intelligent behavior to perform tasks autonomously. The process involves several steps that help machines learn, make decisions and improve over time:Data Collection: AI systems rely on large sets of data which could include images, text or sensor readings. For example, teaching an AI to recognize cats, we collect a dataset of labeled cat images.Processing and Learning: It uses algorithms to analyze data and'

***Retrieval Chain***

- *`ChatOpenAI` is LangChain’s wrapper around OpenAI’s chat models (like GPT‑4, GPT‑4o, GPT‑3.5).*
- *It provides a standardized interface so you can use OpenAI models inside LangChain pipelines (chains, agents, tools).*

***Initialize the LLM***

In [10]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o") # Creates an instance of the GPT‑4o model (OpenAI’s multimodal GPT‑4 “omni” model).
print(llm) # Displays the configuration of the ChatOpenAI object (e.g., which model is loaded, temperature, API settings).

profile={'max_input_tokens': 128000, 'max_output_tokens': 16384, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True} client=<openai.resources.chat.completions.completions.Completions object at 0x135f22d70> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x1407c7430> root_client=<openai.OpenAI object at 0x135f22b90> root_async_client=<openai.AsyncOpenAI object at 0x1407c5540> model_name='gpt-4o' model_kwargs={} openai_api_key=SecretStr('**********') stream_usage=True


***Define the Prompt Template***

- *`ChatPromptTemplate` - LangChain utility for building structured prompts for chat models. Allows to define multiple roles (system, user, assistant) and message templates.*
- *`{context}` will be dynamically filled with retrieved documents.*
- *This template instructs the LLM to only use the provided context when answering*

In [11]:
## Retrieval Chain, Document chain

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>


"""
)

***Create the Document Chain***

- *A document Chain takes document and a prompt and passes them to the LLM.*
- *Stuff means all retrieved documents are stuffed into the {context} placeholder*

In [12]:
# create a document chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain # Importing the function to create a document chain that combines documents using a specified method
document_chain = create_stuff_documents_chain(llm,prompt) # Creating a document chain using the specified language model and prompt template
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
| ChatOpenAI(profile={'max_input_tokens': 128000, 'max_output_tokens': 16384, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}, client=<openai.resources.chat.completions.co

In [13]:
# from langchain_core.documents import Document
# document_chain.invoke({
#     "input":"LangSmith has two usage limits: total traces and extended",
#     "context":[Document(page_content="LangSmith has two usage limits: total traces and extended traces. These correspond to the two metrics we've been tracking on our usage graph. ")]
# })

***Set up Retrieve from FAISS***

In [14]:
retriever = vectorstoredb.as_retriever()

***Create a retrieval Chain***

- *This chain first uses the retriever to fetch documents.*
- *Then it passes those documents into the document chain(with the prompt + LLM).*
- *This is the classic RAG setup.*

In [15]:
from langchain_classic.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever,document_chain)

In [16]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x1407c5480>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
            | ChatOpe

In [17]:
## Get the response form the LLM
response=retrieval_chain.invoke({"input":"What is Natural Language Processing?"})
response['answer']

'Based on the provided context, can you explain the key difference between traditional AI and generative AI?  \n\nTraditional AI primarily focuses on analyzing and classifying data, recognizing patterns, and making predictions based on the input data. It typically involves processing data to identify existing characteristics or categories. On the other hand, Generative AI goes a step further by using learned patterns from large datasets to create new, original outputs. This involves generating content such as text, images, music, or video, effectively moving beyond recognition to creation.'

In [18]:

response

{'input': 'What is Natural Language Processing?',
 'context': [Document(id='f82464ab-55f7-44e7-9a63-22aeebe786e6', metadata={'source': 'https://www.geeksforgeeks.org/artificial-intelligence/what-is-artificial-intelligence-ai/', 'title': 'What is Artificial Intelligence (AI) - GeeksforGeeks', 'description': 'Your All-in-One Learning Portal: GeeksforGeeks is a comprehensive educational platform that empowers learners across domains-spanning computer science and programming, school education, upskilling, commerce, software tools, competitive exams, and more.', 'language': 'en-US'}, page_content='uses algorithms to identify patterns within data and improve its performance over time without human intervention.2. Generative AIGenerative AI is designed to create new content whether it\'s text, images, music or video. Unlike traditional AI which typically focuses on analyzing and classifying data, it goes a step further by using patterns it has learned from large datasets to generate new origi

In [19]:
response['context']

[Document(id='f82464ab-55f7-44e7-9a63-22aeebe786e6', metadata={'source': 'https://www.geeksforgeeks.org/artificial-intelligence/what-is-artificial-intelligence-ai/', 'title': 'What is Artificial Intelligence (AI) - GeeksforGeeks', 'description': 'Your All-in-One Learning Portal: GeeksforGeeks is a comprehensive educational platform that empowers learners across domains-spanning computer science and programming, school education, upskilling, commerce, software tools, competitive exams, and more.', 'language': 'en-US'}, page_content='uses algorithms to identify patterns within data and improve its performance over time without human intervention.2. Generative AIGenerative AI is designed to create new content whether it\'s text, images, music or video. Unlike traditional AI which typically focuses on analyzing and classifying data, it goes a step further by using patterns it has learned from large datasets to generate new original outputs. It "creates" rather than just "recognizes."3. Nat